<a href="https://colab.research.google.com/github/jishajoble/machine-learning/blob/master/TCh_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mpl_toolkits
%matplotlib inline
import csv
import time
pd.set_option('display.max_columns', 600)
pd.set_option('display.width', 1000)
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
file_path = '/content/toronto_train.csv'
# time taken to read data
# s_time_chunk = time.time()
df = pd.read_csv(file_path)
# chunk = pd.read_csv(file_path,on_bad_lines='skip')
# e_time_chunk = time.time()
  
# print("With chunks: ", (e_time_chunk-s_time_chunk), "sec")
# df = pd.concat(chunk)


df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10,37,66,70,71,84,85,86,87,100) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if sys.path[0] == '':


In [4]:
for column in df.columns: 
    if df[column].count() <= (df.shape[0])*.5 :
        df=df.drop(column, axis=1) 
df.shape
not_important = ['occu','com_coopb','map_page','pix_updt','cross_street','street','zip','full_address','ml_num','del',
                  'feed_idx','feed_vow','feed_ddf','added_ts','brokerage','created_at.1','updated_at.1','sold_date',
                  'del.1','bit','ar_uid.1','created_at','created_at','tax_year','board','display','slug','updated_at']
same = ['ar_uid','city','area','province','listing_status','sale_lease','building_type','prop_type','building_type',]
df=df.drop(not_important, axis=1) 
df=df.drop(same, axis=1) 
later = ['a_c','status','heat_source','heat_type','water','fronting']
df=df.drop(later, axis=1) 
df.shape

(16265, 24)

In [5]:
from datetime import datetime
df['updated_date'] = [d.split(' ')[0] for d in df['updated_ts']]
df['updated_date'] = pd.to_datetime(df['updated_date'])
estimated_date = df['updated_date'].max()
estimated_date = pd.to_datetime(estimated_date)
# print(df['updated_date'])
df['days_af_update'] = (df['updated_date'] - estimated_date)/np.timedelta64(1, 'D')
# print(df['days_af_update'])
clear_date = ['updated_date','updated_ts']
df=df.drop(clear_date, axis=1) 

df.shape

(16265, 24)

In [6]:
scale_mapper = {"Fin W/O":7, "Apartment":6, "Finished":5,"Part Fin":4,"Full":3,"Sep Entrance":3,"Unfinished":2,"other":1,"Crawl Space":1,"Half":1,"None":1,"Other":1,"Part Bsmt":1,"W/O":1,"W/O":1,"Walk-Up":1}
df['basement'] = df['basement'].replace(scale_mapper)
scale_mapper = {"Attached":5, "Built-In":5,"Detached":4,"Carport":3,"Other":2,"None":1}
df["garage_type"] = df["garage_type"].replace(scale_mapper)
scale_mapper = {"Indoor":5, "Inground":4,"Outdoor":3,"Abv Grnd":2,"None":1}
df["pool"] = df["pool"].replace(scale_mapper)
scale_mapper = {"2-Storey":5, "1 1/2 Storey":4,"2 1/2 Storey":4,"Outdoor":3,"3-Storey":3,"Bungaloft":2,"Bungalow":2,"Bungalow-Raised":2,"Multi-Level":1,"Sidesplit 3":1,"Other":1,"Backsplit 3":1,"Backsplit 4":1,"Backsplit 5":1,"Backsplt-All":1,
                 "Multi-Level":1,"Sidesplit 3":1,"Sidesplit 4":1,"Sidesplit 5":1,"Sidesplt-All":1}
df["building_style"] = df["building_style"].replace(scale_mapper)
scale_mapper = {"Y":2,"N":1}
df["disp_addr"] = df["disp_addr"].replace(scale_mapper)
scale_mapper = {"Y":2,"N":1}
df["family_room"] = df["family_room"].replace(scale_mapper)
scale_mapper = {"Pvt Double":7, "Private":6,"Front Yard":5,"Circular":4,"Lane":3,"Other":2,"Available":2,"Rt-Of-Way":2,
                 "Mutual":2,"None":1}
df["drive"] = df["drive"].replace(scale_mapper)


In [7]:
#droping values with missing rows
df = df[df['sold_price'].notna()]
# df = df[df['sold_date'].notna()]
# df = df[df['lot_size'].notna()]
df = df[df['bed'].notna()]
df= df[df['sold_price'] != 0]
df= df[df['bed'] != 0]
df= df[df['lat'] != 0]
df= df[df['kitchen'] != 0]
df = df[df['lot_size'].notna()]

df.shape
df = df.dropna(subset=['lot_size'])

# df.groupby('fronting')['fronting'].agg('count')
df2 = df.copy()  



In [8]:
features_with_na=[features for features in df.columns if df[features].isnull().sum()>1]
features_with_na

[]

In [9]:
# this is mandatory for working this section. don't delete it
df2['lot_size'] = df2['lot_size'].astype(str)
df2['lot_size']=df2['lot_size'].apply(lambda x:(x.replace("Feet","").replace("Metres","").replace("Acres","").replace("F","").replace("   ","x")))
# print(df2['lot_size'])
df2 = df2.reset_index(drop = True)

df2['length'] = pd.to_numeric(df2['lot_size'].apply(lambda x: ((x.lstrip().split('x')[0]).split(' ')[0]) ))
df2['breadth'] = pd.to_numeric(df2['lot_size'].apply(lambda x: ((x.lstrip().split('x')[1]).split(' ')[1]) ))
# print(df2['length'])
# print(df2['breadth'])
df2['lot_sqft'] = df2['length']*df2['breadth']
# print(df2['lot_sqft'])
clear  = ['lot_size','length','breadth']
df2 = df2.drop(clear,axis=1)

In [10]:
df_addr = df2.copy()
df3 = df_addr.drop(['addr'],axis=1)


In [11]:
columns = ['community']

In [ ]:
# df['lat'].isin([0]).any().any() #True
# df['bed'].isin([0]).any().any() #True
# df['bed_plus'].isin([0]).any().any() #True
# df['lot_size'].isin([0]).any().any() #True
# df.groupby('heat_source')['heat_source'].agg('count')
# df.groupby('heat_type')['heat_type'].agg('count')
# df.groupby('water')['water'].agg('count')





In [12]:
def category_onehot_multcols(multcolumns):
    df4_final = df3
    i=0
    for fields in multcolumns:   
        print(fields)
        df4 = pd.get_dummies(df3[fields],drop_first=True)
        
        df3.drop([fields],axis=1,inplace=True)
        if i==0:
            df4_final=df4.copy()
        else:
            df4_final=pd.concat([df4_final,df4],axis=1)
        i=i+1     
    df4_final=pd.concat([df3,df4_final],axis=1)
        
    return df4_final

df4 = category_onehot_multcols(columns)

final_df = df4.copy()
final_df

community


,lat,long,listed_price,sold_price,bed,bed_plus,bath,building_style,parking_space,garage_space,garage_type,basement,pool,kitchen,disp_addr,family_room,drive,tax,lot_depth,lot_front,days_af_update,lot_sqft,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Crescent Town,Danforth,Danforth Village-East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,East York,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights,Humber Summit,Humberlea-Pelmo Park W4,Humberlea-Pelmo Park W5,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,L'Amoreaux,Lambton Baby Point,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico,Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,O'Connor-Parkview,Oakridge,Oakwood-Vaughan,Palmerston-Little Italy,Parkwoods-Donalda,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge E10,Rouge E11,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St. Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,The Beaches,Thistletown-Beaumonde Heights,Trinity-Bellwoods,University,Victoria Village,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pellam Park,Wexford-Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St. Clair,York University Heights,Yorkdale-Glen Park
0,43.731145,-79.362521,14850000.0,13800000.0,6.0,0,8,2,10,2.0,5,5,5,1,2,2,4,43388.00,299.00,322.00,-73.0,96278.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,45.668813,-67.092961,13999000.0,13000000.0,4.0,1,9,2,16,4.0,5,7,4,2,2,2,4,50123.00,428.43,302.08,-79.0,129420.1344,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,43.774665,-79.437204,2688000.0,2500000.0,4.0,1,6,1,6,2.0,4,6,1,1,2,2,7,9889.00,132.00,60.00,-47.0,7920.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,43.743967,-79.438254,2780000.0,2780000.0,4.0,1,5,5,4,2.0,5,5,1,1,2,2,7,11731.45,120.00,50.00,-58.0,6000.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [13]:
final_df =final_df.loc[:,~final_df.columns.duplicated()]
df_addr =df_addr.loc[:,~df_addr.columns.duplicated()]
final_df.shape

(16046, 157)

In [14]:
import xgboost
classifier=xgboost.XGBRegressor()

In [15]:
import xgboost
regressor=xgboost.XGBRegressor()
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]

In [16]:
labels = final_df['sold_price']
from sklearn.model_selection import train_test_split
train1 = final_df.drop(['sold_price'],axis=1)
x_train , x_test , y_train , y_test = train_test_split(train1, labels , test_size = 0.20,random_state =2)
y_test

4771     1130000.0
1755     1570333.0
11358     912000.0
13723    1200000.0
2973     1616666.0
           ...    
3022     1080000.0
12319    1350000.0
6626      815000.0
11730    1570000.0
7484     1660000.0
Name: sold_price, Length: 3210, dtype: float64

In [17]:
## Hyper Parameter Optimization


n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [18]:
# Set up the random search with 4-fold cross validation
random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [20]:
final_df.describe(include = 'all')
x_train

,lat,long,listed_price,bed,bed_plus,bath,building_style,parking_space,garage_space,garage_type,basement,pool,kitchen,disp_addr,family_room,drive,tax,lot_depth,lot_front,days_af_update,lot_sqft,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Crescent Town,Danforth,Danforth Village-East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,East York,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights,Humber Summit,Humberlea-Pelmo Park W4,Humberlea-Pelmo Park W5,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,L'Amoreaux,Lambton Baby Point,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico,Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,O'Connor-Parkview,Oakridge,Oakwood-Vaughan,Palmerston-Little Italy,Parkwoods-Donalda,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge E10,Rouge E11,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St. Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,The Beaches,Thistletown-Beaumonde Heights,Trinity-Bellwoods,University,Victoria Village,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pellam Park,Wexford-Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St. Clair,York University Heights,Yorkdale-Glen Park
4241,43.799823,-79.260406,2488800.0,5.0,2,7,5,4,2.0,5,5,1,1,2,2,7,8884.13,167.31,31.53,-142.0,5275.2843,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14288,43.690906,-79.352872,1299900.0,4.0,0,5,3,1,1.0,5,5,1,1,2,2,6,7478.80,150.00,20.34,-71.0,3051.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7891,43.603571,-79.517376,899000.0,3.0,0,2,4,1,0.0,1,4,1,1,2,2,2,3430.31,100.04,25.03,-247.0,2504.0012,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6009,43.728358,-79.389683,5598000.0,4.0,1,5,5,4,1.0,5,5,1,1,2,2,7,14968.61,90.00,52.00,-240.0,4680.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [21]:
random_cv.fit(x_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[15:10:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=5, estimator=XGBRegressor(), n_iter=50, n_jobs=4,
                   param_distributions={'base_score': [0.25, 0.5, 0.75, 1],
                                        'booster': ['gbtree', 'gblinear'],
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2],
                                        'max_depth': [2, 3, 5, 10, 15],
                                        'min_child_weight': [1, 2, 3, 4],
                                        'n_estimators': [100, 500, 900, 1100,
                                                         1500]},
                   random_state=42, return_train_score=True,
                   scoring='neg_mean_absolute_error', verbose=5)

In [22]:
random_cv.best_estimator_

XGBRegressor(base_score=0.25, max_depth=10, min_child_weight=3)

In [23]:
random_cv.best_estimator_

XGBRegressor(base_score=0.25, max_depth=10, min_child_weight=3)

In [24]:
regressor=xgboost.XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=10, min_child_weight=3, missing=None, n_estimators=900,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [26]:
regressor.fit(x_train,y_train)

XGBRegressor(base_score=0.25, max_depth=10, min_child_weight=3,
             n_estimators=900, silent=True)

In [27]:
import pickle
filename = 'finalized_model_feb12.pkl'
pickle.dump(classifier, open(filename, 'wb'))

In [28]:
print(x_train.shape)
print(x_test)

(12836, 156)
             lat       long  listed_price  bed  bed_plus  bath  building_style  parking_space  garage_space  garage_type  basement  pool  kitchen  disp_addr  family_room  drive      tax  lot_depth  lot_front  days_af_update   lot_sqft  Agincourt South-Malvern West  Alderwood  Annex  Banbury-Don Mills  Bathurst Manor  Bayview Village  Bayview Woods-Steeles  Bedford Park-Nortown  Beechborough-Greenbrook  Bendale  Birchcliffe-Cliffside  Black Creek  Blake-Jones  Briar Hill-Belgravia  Bridle Path-Sunnybrook-York Mills  Broadview North  Brookhaven-Amesbury  Cabbagetown-South St. James Town  Caledonia-Fairbank  Casa Loma  Centennial Scarborough  Church-Yonge Corridor  Clairlea-Birchmount  Clanton Park  Cliffcrest  Corso Italia-Davenport  Crescent Town  Danforth  Danforth Village-East York  Don Valley Village  Dorset Park  Dovercourt-Wallace Emerson-Junction  Downsview-Roding-CFB  Dufferin Grove  East End-Danforth  East York  Edenbridge-Humber Valley  Eglinton East  Elms-Old Rexd

In [36]:
# x_test = [[43.774167,-79.205956,3.0,0,2,1,2,1.0,4,5,1,1,2,11,2,2858.68,120.00,80.00],[43.774167,-79.205956,3.0,0,2,1,2,1.0,4,5,1,1,2,11,2,2858.68,120.00,80.00]]
# y_pred=regressor.predict(x_test)

print(x_test.shape)
addr_y = df_addr['sold_price']
y_addr = addr_y[0:3210]
addr_test = train1.iloc[0:3210,0:157]
addr_final = df_addr.iloc[0:3210,0:157]
print(addr_test.shape)

y_pred = regressor.predict(x_test)
x_test_copy = x_test.copy()
diff_addr = (y_pred-y_test).abs()
print(addr_y.shape)
print(y_pred.shape)
print(diff_addr)


(3210, 156)
(3210, 156)
(16046,)
(3210,)
4771      92295.6250
1755     157219.0000
11358    106001.8750
13723     75147.6250
2973      26825.1250
            ...     
3022     119245.1250
12319     10245.3750
6626      57536.9375
11730     21245.6250
7484     235501.5000
Name: sold_price, Length: 3210, dtype: float64


In [43]:
# **********************************XG-boost with out address*****************************************
import csv
fields = 'addr','lat','long','bed','bed_plus','bath','building_style','parking_space','garage_space','garage_type','basement','pool','kitchen','disp_addr','del','family_room','drive','kitchen_plus','tax','lot_depth','lot_front','days_af_update','lot_sqft','listed_price','sold_price','estimated_price','diff','diff_jj'
print('fields',fields)
# print(x_test)
# name of csv file 
filename = "estimated_xgboost_12.csv"
print(x_test_copy.shape)
print(len(y_pred))
x_test_copy['sold_price']=y_test
x_test_copy['estimated_price']=y_pred
x_test_copy['diff']=(y_pred-y_test)
x_test_copy['diff_jj']=(y_pred-y_test).abs()
estimated = pd.DataFrame(x_test_copy, columns=fields)
estimated.to_csv('estimated_xgboost_12.csv',' ')
print(len(x_test_copy['diff_jj']))
(sum(x_test_copy['diff_jj']))/len(x_test_copy['diff_jj'])


fields ('addr', 'lat', 'long', 'bed', 'bed_plus', 'bath', 'building_style', 'parking_space', 'garage_space', 'garage_type', 'basement', 'pool', 'kitchen', 'disp_addr', 'del', 'family_room', 'drive', 'kitchen_plus', 'tax', 'lot_depth', 'lot_front', 'days_af_update', 'lot_sqft', 'listed_price', 'sold_price', 'estimated_price', 'diff', 'diff_jj')
(3210, 161)
3210
3210


88320.71718457945

In [ ]:
import csv
fields = 'addr','lat','long','bed','bed_plus','bath','building_style','parking_space','garage_space','garage_type','basement','pool','kitchen','del','family_room','drive','tax','lot_depth','lot_front','days_af_update','lot_sqft','listed_price','sold_price','estimated_price','diff','diff_jj'
print('fields',fields)
# print(x_test)
# name of csv file 
filename = "estimated_xgboost_addr_feb12.csv"
print(x_test.shape)
print(len(y_pred))
# x_test['estimated_price']=y_pred
# x_test['diff']=(y_pred-y_test).abs()
addr_final['estimated_price']=y_pred.astype(int)
addr_final['diff']=((diff_addr).abs()).astype(int)
estimated = pd.DataFrame(addr_final, columns=fields)
estimated.to_csv('estimated_xgboost_addr_feb12.csv',' ')

In [ ]:
(sum(addr_final['diff']))/len(addr_final['diff'])
addr_final['diff'].shape

In [31]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
x_train_l , x_test_l , y_train_l , y_test_l = train_test_split(train1 , labels , test_size = 0.20,random_state =2)
reg.fit(x_train_l,y_train_l)
reg.score(x_test_l,y_test_l)

0.9785333263677406

In [ ]:
import csv
fields = 'addr','lat','long','listed_price','bed','bed_plus','bath','building_style','parking_space','garage_space','garage_type','basement','pool','kitchen','disp_addr','family_room','drive','kitchen_plus','tax','lot_depth','lot_front','days_af_update','lot_sqft','estimated_price','diff','diff_xgboost' 
print('fields',fields)
# print(x_test)
# name of csv file 
filename = "estimated_linear.csv"
print(x_test_l.shape)
print(len(y_pred))
y_pred_l = reg.predict(x_test_l)
x_test_l['estimated_price']=y_pred_l
x_test_l['diff_linear']=(y_pred_l-y_test_l).abs()
x_test_l['diff_xgboost']=(y_pred-y_test).abs()
estimated = pd.DataFrame(x_test_l, columns=fields)
estimated.to_csv('estimated_linear.csv')

fields ('addr', 'lat', 'long', 'listed_price', 'bed', 'bed_plus', 'bath', 'building_style', 'parking_space', 'garage_space', 'garage_type', 'basement', 'pool', 'kitchen', 'disp_addr', 'family_room', 'drive', 'kitchen_plus', 'tax', 'lot_depth', 'lot_front', 'days_af_update', 'lot_sqft', 'estimated_price', 'diff', 'diff_xgboost')
(3210, 157)
3210
